In [2]:
import java.util.{Locale, UUID}
import java.nio.file.Paths

import java.util.{Locale, UUID}
import java.nio.file.Paths


In [3]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.SparkContext
import org.apache.spark.graphx.{Edge, Graph, VertexId}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.expressions.{UserDefinedFunction, Window}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.log4j.{Level, Logger}
import org.apache.spark.SparkContext
import org.apache.spark.graphx.{Edge, Graph, VertexId}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.expressions.{UserDefinedFunction, Window}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, SparkSession}


## Initialisation de la session spark

In [4]:
val spark = SparkSession.builder().appName("GraphsKlouviRiva").getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5ca78fb5


## Initialisation quelques variables

In [5]:
//val input_file_path = "c:/users/Fabrice/Documents/MIASD/Graphs/"
val input_file_path = "file://" + Paths.get(".").toAbsolutePath + "/"
var delimiter = ","  // à modifier en fonction de la structure du fichier CSV

input_file_path: String = file:///home/masterai/dev/master_iasd/graph/project/./
delimiter: String = ,


## Quelques fonctions UDF utiles pour la transformation

In [6]:
val fullnameUdf: UserDefinedFunction = udf((col1: String, col2: String) => {
    col1.concat(" ").concat(col2).toUpperCase(Locale.FRANCE)
  })

val authorArrayUdf: UserDefinedFunction = udf((col1: String) => {
    col1.toUpperCase(Locale.FRANCE).split(",")
  })

val yearUdf: UserDefinedFunction = udf((col1: String) => {
    col1.split("-")(0).toLong
  })

val labUdf: UserDefinedFunction = udf((col1: String) => {
    if (col1 == null) "EXT" else "LAMSADE"
  })

fullnameUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function2>,StringType,Some(List(StringType, StringType)))
authorArrayUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,ArrayType(StringType,true),Some(List(StringType)))
yearUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,LongType,Some(List(StringType)))
labUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


## Importation de la liste des membres du LAMSADE

In [7]:
var members = spark.read
  .option("header","false")
  .option("delimiter", delimiter)
  .option("inferSchema", "true")
  .csv(input_file_path + "Membres.csv")
  .withColumn("fullname", fullnameUdf($"_c0", $"_c1"))
    .drop("_c0").drop("_c1")
  .cache()

members.show(10, truncate = false)


+-----------------+
|fullname         |
+-----------------+
|STEPHANE AIRIAU  |
|HASSAN AISSI     |
|JAMAL ATIF       |
|CRISTINA BAZGAN  |
|KHALID BELHAJJAME|
|MARIE-JO BELLOSTA|
|DENIS BOUYSSOU   |
|OLIVIER CAILLOUX |
|TRISTAN CAZENAVE |
|YANN CHEVALEYRE  |
+-----------------+
only showing top 10 rows



members: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [fullname: string]


## Importation des publications

In [8]:
var publications = spark.read.format("csv")
    .option("header", "true")
    .option("delimiter", delimiter)
    .option("inferSchema", "true")
    .csv(input_file_path + "publications_old.csv")
    .withColumn("authors", authorArrayUdf($"authFullName_s"))
    .withColumn("year", yearUdf($"producedDate_s"))
    .withColumn("author", explode($"authors"))
    .drop("docType_s").drop("title_s").drop("authFullName_s").drop("producedDate_s")
    .drop("domain_s").drop("journalTitle_s").drop("conferenceTitle_s").drop("fr_keyword_s")
    .drop("labStructAcronym_s").drop("authors")
    .cache()

publications.show(10, truncate = false)

+------------+----+---------------+
|halId_s     |year|author         |
+------------+----+---------------+
|hal-02012443|2018|ERIC BENHAMOU  |
|hal-02012443|2018|BEATRICE GUEZ  |
|hal-02074110|2017|FABIEN LABERNIA|
|hal-02074110|2017|BRUNO ZANUTTINI|
|hal-02074110|2017|BRICE MAYAG    |
|hal-02074110|2017|FLORIAN YGER   |
|hal-02074110|2017|JAMAL ATIF     |
|hal-01619969|2017|FABIEN LABERNIA|
|hal-01619969|2017|BRUNO ZANUTTINI|
|hal-01619969|2017|BRICE MAYAG    |
+------------+----+---------------+
only showing top 10 rows



publications: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [halId_s: string, year: bigint ... 1 more field]


## Création de la liste des auteurs

Nous commençons par assigner aux auteurs leur laboratoire d'origine en croisant la liste des publications (avec leurs auteurs) avec celle des membres de LAMSADE. Par convention, nous assignons le laboratoire "EXT" aux auteurs non membres de LAMSADE. Leur véritable laboratoire d'origine n'a d'importance pour l'analyse

In [9]:
val window = Window.orderBy("author")
val authors = publications.select("author").join(members.select("fullname"),
  publications("author") === members("fullname"), "left")
  .withColumn("lab", labUdf($"fullname")).distinct()
  .withColumn("author_id", row_number().over(window)) // On assigne à chaque auteur un idenfiant (author_id)
  .drop("fullname")
authors.show(truncate = false)

+--------------------+---+---------+
|author              |lab|author_id|
+--------------------+---+---------+
|ABDALLAH SAFFIDINE  |EXT|1        |
|ABHINAV GUPTA       |EXT|2        |
|ADEL BIBI           |EXT|3        |
|ADRIAN HARET        |EXT|4        |
|ADRIAN LECOUTRE     |EXT|5        |
|AGNÈS RICO          |EXT|6        |
|AHLAM AZZAMOURI     |EXT|7        |
|AHMED SAMET         |EXT|8        |
|AKIHISA SONODA      |EXT|9        |
|ALAIN HERTZ         |EXT|10       |
|ALAIN RAKOTOMAMONJY |EXT|11       |
|ALAN LUKEŽIČ        |EXT|12       |
|ALBAN GAIGNARD      |EXT|13       |
|ALBERTO CAPRARA     |EXT|14       |
|ALBERTO CESELLI     |EXT|15       |
|ALBERTO COLORNI     |EXT|16       |
|ALBERTO SANTINI     |EXT|17       |
|ALDO PAGANO         |EXT|18       |
|ALESSANDRO SOLIMANDO|EXT|19       |
|ALEXANDRE ARAUJO    |EXT|20       |
+--------------------+---+---------+
only showing top 20 rows



window: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@11c445bc
authors: org.apache.spark.sql.DataFrame = [author: string, lab: string ... 1 more field]


## Mise à jours des publications avec le laboratoire de leurs auteurs

On fait une autre jointure (join()) entre  `publications` et `authors` pour créer la relation entre une publication et son auteur (contenant également son labo).

In [10]:

val publicationLab = publications.join(authors, publications("author") === authors("author"), "left")
  .drop(authors("author"))

publicationLab.show(20, truncate = false)

+--------------+----+------------------+---+---------+
|halId_s       |year|author            |lab|author_id|
+--------------+----+------------------+---+---------+
|hal-01495356  |2015|FLORENT MONTIGNAC |EXT|169      |
|hal-01336773  |2015|KAI XUE           |EXT|294      |
|lirmm-02342803|2018|MARIA SERNA       |EXT|347      |
|hal-01336773  |2015|ANDRÉS MONTERO    |EXT|40       |
|hal-01653334  |2017|BINLONG LI        |EXT|69       |
|hal-01715095  |2017|DIMITRIOS XEFTERIS|EXT|124      |
|hal-02308114  |2019|OKSANA SYTAR      |EXT|432      |
|hal-01740137  |2017|BASILE CLEMENT    |EXT|58       |
|hal-01680527  |2017|PHILIPPE CAILLOU  |EXT|454      |
|hal-02310662  |2019|HENNING FERNAU    |EXT|213      |
|hal-02184942  |2019|HENNING FERNAU    |EXT|213      |
|hal-02104830  |2018|LYDIA TLILANE     |EXT|331      |
|hal-02303794  |2019|PETER BIRO        |EXT|451      |
|hal-01369835  |2016|FABIEN MATHIEU    |EXT|155      |
|hal-01369877  |2016|FABIEN MATHIEU    |EXT|155      |
|hal-01632

publicationLab: org.apache.spark.sql.DataFrame = [halId_s: string, year: bigint ... 3 more fields]


## Création de la liste des coauteurs

A l'aide d'une auto-jointure de la table `publications`, on déduit le couple d'auteurs ayant contribué sur une même publication. Lorsqu'une publication a été réalisée par un seul auteur, les champs relatifs `author2` sont null

In [11]:
var coauthors = publicationLab.as("pub1").join(publicationLab.as("pub2"),
  col("pub1.halId_s") === col("pub2.halId_s")
    && col("pub1.author") < col("pub2.author")
  , "left")
  .withColumn("pub_year", $"pub1.year")
  .withColumn("author1_id", $"pub1.author_id")
  .withColumn("author2_id", $"pub2.author_id")
  .withColumn("author1", $"pub1.author")
  .withColumn("author2", $"pub2.author")
  .withColumn("lab1", $"pub1.lab")
  .withColumn("lab2", $"pub2.lab")

coauthors = coauthors.drop($"pub1.author").drop($"pub2.author")
  .drop($"pub1.author_id").drop($"pub2.author_id")
  .drop($"pub1.year").drop($"pub2.halId_s")
  .drop($"pub1.lab").drop($"pub2.lab")
  .drop($"pub1.year").drop($"pub2.year")

coauthors.show(20, truncate = false)

+------------+--------+----------+----------+---------------------+---------------------+-------+-------+
|halId_s     |pub_year|author1_id|author2_id|author1              |author2              |lab1   |lab2   |
+------------+--------+----------+----------+---------------------+---------------------+-------+-------+
|hal-01179503|2017    |456       |488       |PIERRE CARDALIAGUET  |SAEED HADIKHANLOO    |EXT    |EXT    |
|hal-01179503|2017    |488       |null      |SAEED HADIKHANLOO    |null                 |EXT    |null   |
|hal-01837155|2018    |585       |null      |Y. NOACK             |null                 |EXT    |null   |
|hal-01837155|2018    |255       |285       |JEAN-CLAUDE RAYNAL   |JULIETTE ROUCHIER    |EXT    |LAMSADE|
|hal-01837155|2018    |255       |585       |JEAN-CLAUDE RAYNAL   |Y. NOACK             |EXT    |EXT    |
|hal-01837155|2018    |285       |585       |JULIETTE ROUCHIER    |Y. NOACK             |LAMSADE|EXT    |
|hal-01690385|2018    |494       |null      |S

coauthors: org.apache.spark.sql.DataFrame = [halId_s: string, pub_year: bigint ... 6 more fields]
coauthors: org.apache.spark.sql.DataFrame = [halId_s: string, pub_year: bigint ... 6 more fields]


## Création de la relation reliant deux auteurs

Nous allons commencer par enlever les publications qui n'ont pas au moins un couple d'auteurs (`author2=null`). Nous regroupons ensuite les publications pour lesquelles deux coautheurs ont contribué afin de calculer le nombre de publications et l'année du début de leur colaboration

In [12]:
val coauthorsRelationships = coauthors.where("author2 is not null").groupBy($"author1", $"author2")
  .agg(
    min("author1_id").as("author1_id"), min("author2_id").as("author2_id"),
    min("pub_year").as("first_year"), //année du début de colaboration
    count("halId_s").as("pub_count"), // nombre de publications communes
    min("lab1").as("lab1"), min("lab2").as("lab2"))

coauthorsRelationships.show(20, truncate = false)

+------------------+------------------------+----------+----------+----------+---------+----+-------+
|author1           |author2                 |author1_id|author2_id|first_year|pub_count|lab1|lab2   |
+------------------+------------------------+----------+----------+----------+---------+----+-------+
|ABDALLAH SAFFIDINE|BRUNO ZANUTTINI         |1         |75        |2018      |1        |EXT |EXT    |
|ABDALLAH SAFFIDINE|DOROTHEA BAUMEISTER     |1         |130       |2017      |1        |EXT |EXT    |
|ABDALLAH SAFFIDINE|EDOUARD BONNET          |1         |134       |2016      |1        |EXT |EXT    |
|ABDALLAH SAFFIDINE|FLORIAN JAMAIN          |1         |171       |2016      |1        |EXT |EXT    |
|ABDALLAH SAFFIDINE|FRANÇOIS SCHWARZENTRUBER|1         |178       |2018      |1        |EXT |EXT    |
|ABDALLAH SAFFIDINE|HUA-MIN LIANG           |1         |227       |2015      |1        |EXT |EXT    |
|ABDALLAH SAFFIDINE|HUNG-HSUAN LIN          |1         |228       |2015      |1   

coauthorsRelationships: org.apache.spark.sql.DataFrame = [author1: string, author2: string ... 6 more fields]


## Création du graphe

Les noeuds contiennent les autheurs (nom complet + laboratoire) et les propriétés sont caractérisées par le nombre de collaborations et le début de la collaboration

In [13]:
val authorsVertices: RDD[(VertexId, (String, String))] = authors.distinct().rdd.map(row => (row.getInt(2).toLong, (row.getString(0), row.getString(1))))
val relationships: RDD[Edge[(Long, Long)]] = coauthorsRelationships.rdd.map(row => Edge(row.getInt(2).toLong, row.getInt(3).toLong, (row.getLong(4), row.getLong(5))))

val defaultAuthor = authorsVertices.first()._2
val graph = Graph(authorsVertices, relationships, defaultAuthor)

authorsVertices: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, (String, String))] = MapPartitionsRDD[204] at map at <console>:39
relationships: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[(Long, Long)]] = MapPartitionsRDD[250] at map at <console>:40
defaultAuthor: (String, String) = (ABDALLAH SAFFIDINE,EXT)
graph: org.apache.spark.graphx.Graph[(String, String),(Long, Long)] = org.apache.spark.graphx.impl.GraphImpl@6347b46a


## Affichage du graphe

On affichage la relation qui lie chaque auteur

In [14]:
graph.triplets.map(
  triplet => "[" + triplet.srcAttr._1 + ", " + triplet.srcAttr._2 + "]" +
    " has collaborated with [" + triplet.dstAttr._1 + ", " + triplet.dstAttr._2 +
    "] on " + triplet.attr._2 + " papers since " + triplet.attr._1
).collect.foreach(println(_))

[ABDALLAH SAFFIDINE, EXT] has collaborated with [BRUNO ZANUTTINI, EXT] on 1 papers since 2018
[ABDALLAH SAFFIDINE, EXT] has collaborated with [DOROTHEA BAUMEISTER, EXT] on 1 papers since 2017
[ABDALLAH SAFFIDINE, EXT] has collaborated with [EDOUARD BONNET, EXT] on 1 papers since 2016
[ABDALLAH SAFFIDINE, EXT] has collaborated with [FLORIAN JAMAIN, EXT] on 1 papers since 2016
[ABDALLAH SAFFIDINE, EXT] has collaborated with [FRANÇOIS SCHWARZENTRUBER, EXT] on 1 papers since 2018
[ABDALLAH SAFFIDINE, EXT] has collaborated with [HUA-MIN LIANG, EXT] on 1 papers since 2015
[ABDALLAH SAFFIDINE, EXT] has collaborated with [HUNG-HSUAN LIN, EXT] on 1 papers since 2015
[ABDALLAH SAFFIDINE, EXT] has collaborated with [I-CHEN WU, EXT] on 1 papers since 2015
[ABDALLAH SAFFIDINE, EXT] has collaborated with [JÉRÔME LANG, EXT] on 1 papers since 2017
[ABDALLAH SAFFIDINE, EXT] has collaborated with [JÖRG ROTHE, EXT] on 1 papers since 2017
[ABDALLAH SAFFIDINE, EXT] has collaborated with [MARIE-LIESSE CAUWE

[ADEL BIBI, EXT] has collaborated with [FILIZ BUNYAK, EXT] on 1 papers since 2015
[ADEL BIBI, EXT] has collaborated with [GAO ZHU, EXT] on 1 papers since 2015
[ADEL BIBI, EXT] has collaborated with [GEORG NEBEHAY, EXT] on 1 papers since 2015
[ADEL BIBI, EXT] has collaborated with [GUNA SEETHARAMAN, EXT] on 1 papers since 2015
[ADEL BIBI, EXT] has collaborated with [GUSTAV HÄGER, EXT] on 1 papers since 2015
[ADEL BIBI, EXT] has collaborated with [GUSTAVO FERNÁNDEZ, EXT] on 1 papers since 2015
[ADEL BIBI, EXT] has collaborated with [HILKE KIERITZ, EXT] on 1 papers since 2015
[ADEL BIBI, EXT] has collaborated with [HING YAU, EXT] on 1 papers since 2015
[ADEL BIBI, EXT] has collaborated with [HONGDONG LI, EXT] on 1 papers since 2015
[ADEL BIBI, EXT] has collaborated with [HONGGANG QI, EXT] on 1 papers since 2015
[ADEL BIBI, EXT] has collaborated with [HORST BISCHOF, EXT] on 1 papers since 2015
[ADEL BIBI, EXT] has collaborated with [HORST POSSEGGER, EXT] on 1 papers since 2015
[ADEL BIBI, 

[ALAN LUKEŽIČ, EXT] has collaborated with [KIMIN YUN, EXT] on 1 papers since 2015
[ALAN LUKEŽIČ, EXT] has collaborated with [KIN HONG WONG, EXT] on 1 papers since 2015
[ALAN LUKEŽIČ, EXT] has collaborated with [LEI LUO, EXT] on 1 papers since 2015
[ALAN LUKEŽIČ, EXT] has collaborated with [LIANG MA, EXT] on 1 papers since 2015
[ALAN LUKEŽIČ, EXT] has collaborated with [LIPENG KE, EXT] on 1 papers since 2015
[ALAN LUKEŽIČ, EXT] has collaborated with [LONGYIN WEN, EXT] on 1 papers since 2015
[ALAN LUKEŽIČ, EXT] has collaborated with [LUCA BERTINETTO, EXT] on 1 papers since 2015
[ALAN LUKEŽIČ, EXT] has collaborated with [LUKA CEHOVIN, EXT] on 1 papers since 2015
[ALAN LUKEŽIČ, EXT] has collaborated with [MAHDIEH POOTSCHI, EXT] on 1 papers since 2015
[ALAN LUKEŽIČ, EXT] has collaborated with [MARIO MARESCA, EXT] on 1 papers since 2015
[ALAN LUKEŽIČ, EXT] has collaborated with [MARTIN DANELLJAN, EXT] on 1 papers since 2015
[ALAN LUKEŽIČ, EXT] has collaborated with [MATEJ KRISTAN, EXT] on 1 

[ALEŠ LEONARDIS, EXT] has collaborated with [LIANG MA, EXT] on 1 papers since 2015
[ALEŠ LEONARDIS, EXT] has collaborated with [LIPENG KE, EXT] on 1 papers since 2015
[ALEŠ LEONARDIS, EXT] has collaborated with [LONGYIN WEN, EXT] on 1 papers since 2015
[ALEŠ LEONARDIS, EXT] has collaborated with [LUCA BERTINETTO, EXT] on 1 papers since 2015
[ALEŠ LEONARDIS, EXT] has collaborated with [LUKA CEHOVIN, EXT] on 1 papers since 2015
[ALEŠ LEONARDIS, EXT] has collaborated with [MAHDIEH POOTSCHI, EXT] on 1 papers since 2015
[ALEŠ LEONARDIS, EXT] has collaborated with [MARIO MARESCA, EXT] on 1 papers since 2015
[ALEŠ LEONARDIS, EXT] has collaborated with [MARTIN DANELLJAN, EXT] on 1 papers since 2015
[ALEŠ LEONARDIS, EXT] has collaborated with [MATEJ KRISTAN, EXT] on 1 papers since 2015
[ALEŠ LEONARDIS, EXT] has collaborated with [MEI WEN, EXT] on 1 papers since 2015
[ALEŠ LEONARDIS, EXT] has collaborated with [MENGDAN ZHANG, EXT] on 1 papers since 2015
[ALEŠ LEONARDIS, EXT] has collaborated wit

[ALI RIDHA MAHJOUB, LAMSADE] has collaborated with [LUIDI SIMONETTI, EXT] on 1 papers since 2019
[ALI RIDHA MAHJOUB, LAMSADE] has collaborated with [MICHAEL POSS, EXT] on 1 papers since 2019
[ALVARO GARCIA-MARTIN, EXT] has collaborated with [AMIR SAFFARI, EXT] on 1 papers since 2015
[ALVARO GARCIA-MARTIN, EXT] has collaborated with [ANDRÉS MONTERO, EXT] on 1 papers since 2015
[ALVARO GARCIA-MARTIN, EXT] has collaborated with [ANTON VARFOLOMIEIEV, EXT] on 1 papers since 2015
[ALVARO GARCIA-MARTIN, EXT] has collaborated with [ATILLA BASKURT, EXT] on 1 papers since 2015
[ALVARO GARCIA-MARTIN, EXT] has collaborated with [BAOJUN ZHAO, EXT] on 1 papers since 2015
[ALVARO GARCIA-MARTIN, EXT] has collaborated with [BERNARD GHANEM, EXT] on 1 papers since 2015
[ALVARO GARCIA-MARTIN, EXT] has collaborated with [BOHYUNG HAN, EXT] on 1 papers since 2015
[ALVARO GARCIA-MARTIN, EXT] has collaborated with [BRAIS MARTINEZ, EXT] on 1 papers since 2015
[ALVARO GARCIA-MARTIN, EXT] has collaborated with [B

...

[SHI-JIM YEN, EXT] has collaborated with [TRISTAN CAZENAVE, LAMSADE] on 1 papers since 2015
[SILVANO FERRINI, EXT] has collaborated with [STEFANO MORETTI, LAMSADE] on 1 papers since 2017
[SIMON HADFIELD, EXT] has collaborated with [SIWEI LYU, EXT] on 1 papers since 2015
[SIMON HADFIELD, EXT] has collaborated with [SIYI LI, EXT] on 1 papers since 2015
[SIMON HADFIELD, EXT] has collaborated with [SONG-CHUN ZHU, EXT] on 1 papers since 2015
[SIMON HADFIELD, EXT] has collaborated with [STEFAN BECKER, EXT] on 1 papers since 2015
[SIMON HADFIELD, EXT] has collaborated with [STEFAN DUFFNER, EXT] on 1 papers since 2015
[SIMON HADFIELD, EXT] has collaborated with [STEPHEN HICKS, EXT] on 1 papers since 2015
[SIMON HADFIELD, EXT] has collaborated with [STUART GOLODETZ, EXT] on 1 papers since 2015
[SIMON HADFIELD, EXT] has collaborated with [SUNGLOK CHOI, EXT] on 1 papers since 2015
[SIMON HADFIELD, EXT] has collaborated with [THOMAS MAUTHNER, EXT] on 1 papers since 2015
[SIMON HADFIELD, EXT] has c

[SONG-CHUN ZHU, EXT] has collaborated with [XIAOMENG WANG, EXT] on 1 papers since 2015
[SONG-CHUN ZHU, EXT] has collaborated with [XIN LI, EXT] on 1 papers since 2015
[SONG-CHUN ZHU, EXT] has collaborated with [XINCHU SHI, EXT] on 1 papers since 2015
[SONG-CHUN ZHU, EXT] has collaborated with [XU ZHAO, EXT] on 1 papers since 2015
[SONG-CHUN ZHU, EXT] has collaborated with [XUE MEI, EXT] on 1 papers since 2015
[SONG-CHUN ZHU, EXT] has collaborated with [YANG HUA, EXT] on 1 papers since 2015
[SONG-CHUN ZHU, EXT] has collaborated with [YANG LI, EXT] on 1 papers since 2015
[SONG-CHUN ZHU, EXT] has collaborated with [YANG LU, EXT] on 1 papers since 2015
[SONG-CHUN ZHU, EXT] has collaborated with [YAO SHIZENG, EXT] on 1 papers since 2015
[SONG-CHUN ZHU, EXT] has collaborated with [YUEZUN LI, EXT] on 1 papers since 2015
[SONG-CHUN ZHU, EXT] has collaborated with [ZEHUA HUANG, EXT] on 1 papers since 2015
[SONG-CHUN ZHU, EXT] has collaborated with [ZHAOYUN CHEN, EXT] on 1 papers since 2015
[SON

[STUART GOLODETZ, EXT] has collaborated with [WEIMING HU, EXT] on 1 papers since 2015
[STUART GOLODETZ, EXT] has collaborated with [WOLFGANG HÜBNER, EXT] on 1 papers since 2015
[STUART GOLODETZ, EXT] has collaborated with [XIAOMENG WANG, EXT] on 1 papers since 2015
[STUART GOLODETZ, EXT] has collaborated with [XIN LI, EXT] on 1 papers since 2015
[STUART GOLODETZ, EXT] has collaborated with [XINCHU SHI, EXT] on 1 papers since 2015
[STUART GOLODETZ, EXT] has collaborated with [XU ZHAO, EXT] on 1 papers since 2015
[STUART GOLODETZ, EXT] has collaborated with [XUE MEI, EXT] on 1 papers since 2015
[STUART GOLODETZ, EXT] has collaborated with [YANG HUA, EXT] on 1 papers since 2015
[STUART GOLODETZ, EXT] has collaborated with [YANG LI, EXT] on 1 papers since 2015
[STUART GOLODETZ, EXT] has collaborated with [YANG LU, EXT] on 1 papers since 2015
[STUART GOLODETZ, EXT] has collaborated with [YAO SHIZENG, EXT] on 1 papers since 2015
[STUART GOLODETZ, EXT] has collaborated with [YUEZUN LI, EXT] o

[TIEN-TUAN DAO, EXT] has collaborated with [TUAN NHA HOANG, EXT] on 1 papers since 2017
[TOMAS VOJIR, EXT] has collaborated with [TONY PRIDMORE, EXT] on 1 papers since 2015
[TOMAS VOJIR, EXT] has collaborated with [WEIMING HU, EXT] on 1 papers since 2015
[TOMAS VOJIR, EXT] has collaborated with [WOLFGANG HÜBNER, EXT] on 1 papers since 2015
[TOMAS VOJIR, EXT] has collaborated with [XIAOMENG WANG, EXT] on 1 papers since 2015
[TOMAS VOJIR, EXT] has collaborated with [XIN LI, EXT] on 1 papers since 2015
[TOMAS VOJIR, EXT] has collaborated with [XINCHU SHI, EXT] on 1 papers since 2015
[TOMAS VOJIR, EXT] has collaborated with [XU ZHAO, EXT] on 1 papers since 2015
[TOMAS VOJIR, EXT] has collaborated with [XUE MEI, EXT] on 1 papers since 2015
[TOMAS VOJIR, EXT] has collaborated with [YANG HUA, EXT] on 1 papers since 2015
[TOMAS VOJIR, EXT] has collaborated with [YANG LI, EXT] on 1 papers since 2015
[TOMAS VOJIR, EXT] has collaborated with [YANG LU, EXT] on 1 papers since 2015
[TOMAS VOJIR, EXT

[XIN LI, EXT] has collaborated with [YANG HUA, EXT] on 1 papers since 2015
[XIN LI, EXT] has collaborated with [YANG LI, EXT] on 1 papers since 2015
[XIN LI, EXT] has collaborated with [YANG LU, EXT] on 1 papers since 2015
[XIN LI, EXT] has collaborated with [YAO SHIZENG, EXT] on 1 papers since 2015
[XIN LI, EXT] has collaborated with [YUEZUN LI, EXT] on 1 papers since 2015
[XIN LI, EXT] has collaborated with [ZEHUA HUANG, EXT] on 1 papers since 2015
[XIN LI, EXT] has collaborated with [ZHAOYUN CHEN, EXT] on 1 papers since 2015
[XIN LI, EXT] has collaborated with [ZHE CHEN, EXT] on 1 papers since 2015
[XIN LI, EXT] has collaborated with [ZHE ZHANG, EXT] on 1 papers since 2015
[XIN LI, EXT] has collaborated with [ZHENYU HE, EXT] on 1 papers since 2015
[XIN LI, EXT] has collaborated with [ZHIBIN HONG, EXT] on 1 papers since 2015
[XINCHU SHI, EXT] has collaborated with [XU ZHAO, EXT] on 1 papers since 2015
[XINCHU SHI, EXT] has collaborated with [XUE MEI, EXT] on 1 papers since 2015
[XINC

## Exportation du graphe vers Neo4J

In [15]:
import java.io.{File, FileInputStream, FileOutputStream}

// Copie du fichier output
def copyFile(from: String, to: String) = {
    val src = new File(from).listFiles.filter(f => f.getName.endsWith(".csv"))(0)
    println("src: ", src)
    new FileOutputStream(to) getChannel() transferFrom(
      new FileInputStream(src) getChannel, 0, Long.MaxValue )
}

import java.io.{File, FileInputStream, FileOutputStream}
copyFile: (from: String, to: String)Long


In [16]:
val outputPath = input_file_path + "output"

outputPath: String = file:///home/masterai/dev/master_iasd/graph/project/./output


Exportation des auteurs

In [17]:

authors.select($"author_id".as(":ID"), $"author".as("fullname:STRING"), $"lab".as("lab:STRING"), lit("Author").as(":LABEL"))
    .coalesce(1).write
    .option("header", "true")
    .option("delimiter", delimiter)
    .mode("overwrite")
    .format("com.databricks.spark.csv")
    .save(outputPath)

copyFile("output", "authors.csv")


(src: ,output/part-00000-fcc5b5e1-311e-47d8-bbda-093b689a99b4-c000.csv)


res8: Long = 18617


Exportation des coauteurs

In [18]:

coauthorsRelationships.select($"author1_id".as(":START_ID"), $"author2_id".as(":END_ID"), 
                              $"first_year".as("first_year:INT"), $"pub_count".as("pub_count:INT"),
                             lit("coauthor").as(":TYPE"))
    .coalesce(1).write
    .option("header", "true")
    .option("delimiter", delimiter)
    .mode("overwrite")
    .format("com.databricks.spark.csv")
    .save(outputPath)

copyFile("output", "coauthors.csv")

(src: ,output/part-00000-8fafde36-1d01-43e8-a23f-3f9b493b5231-c000.csv)


res9: Long = 220958
